In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
np.random.seed(123)

In [3]:
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

## Utility functions

In [4]:
def save_history(history, fn):
    with open(fn, 'wb') as fw:
        pickle.dump(history.history, fw, protocol=2)

def load_history(fn):
    class Temp():
        pass
    history = Temp()
    with open(fn, 'rb') as fr:
        history.history = pickle.load(fr)
    return history

def plot_compare(history, steps=-1):
    if steps < 0:
        steps = len(history.history['accuracy'])
    accuracy = smooth_curve(history.history['accuracy'][:steps])
    val_accuracy = smooth_curve(history.history['val_accuracy'][:steps])
    loss = smooth_curve(history.history['loss'][:steps])
    val_loss = smooth_curve(history.history['val_loss'][:steps])
    
    plt.figure(figsize=(6, 4))
    plt.plot(loss, c='#0c7cba', label='Train Loss')
    plt.plot(val_loss, c='#0f9d58', label='Val Loss')
    plt.xticks(range(0, len(loss), 5))
    plt.xlim(0, len(loss))
    plt.title('Train Loss: %.3f, Val Loss: %.3f' % (loss[-1], val_loss[-1]), fontsize=12)
    plt.legend()
    
    plt.figure(figsize=(6, 4))
    plt.plot(accuracy, c='#0c7cba', label='Train Acc')
    plt.plot(val_accuracy, c='#0f9d58', label='Val Acc')
    plt.xticks(range(0, len(accuracy), 5))
    plt.xlim(0, len(accuracy))
    plt.title('Train Accuracy: %.3f, Val Accuracy: %.3f' % (accuracy[-1], val_accuracy[-1]), fontsize=12)
    plt.legend()

def smooth_curve(points, factor=0.8):
    smoothed = []
    for point in points:
        if smoothed:
            previous = smoothed[-1]
            smoothed.append(previous * factor + point * (1 - factor))
        else:
            smoothed.append(point)
    return smoothed

## Import the dataset

Clone from the github repo

In [5]:
!git clone https://github.com/dbonacorsi/AML2021

fatal: destination path 'AML2021' already exists and is not an empty directory.


In [6]:
!ls -trl

total 8
drwxr-xr-x 1 root root 4096 May  6 13:44 sample_data
drwxr-xr-x 4 root root 4096 May 21 10:48 AML2021


In [7]:
!ls AML2021

AML_Adv_1_DT.ipynb	  AML_Adv_4_NNintro.ipynb     datasets
AML_Adv_2_SVM.ipynb	  AML_Adv_5_MNIST_noNN.ipynb
AML_Adv_3_Ensemble.ipynb  AML_Adv_6_MNIST_CNN.ipynb


In [8]:
!ls AML2021/datasets/

cats_and_dogs_small


In [9]:
!pwd

/content


# Data generator

In [10]:
base_dir = '/content/AML2021/datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Model creation

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_1', 
                 input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2), name='maxpool_1'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(MaxPooling2D((2, 2), name='maxpool_2'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_3'))
model.add(MaxPooling2D((2, 2), name='maxpool_3'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_4'))
model.add(MaxPooling2D((2, 2), name='maxpool_4'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', name='dense_1'))
model.add(Dense(256, activation='relu', name='dense_2'))
model.add(Dense(1, activation='sigmoid', name='output'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=20, 
                              validation_data=validation_generator, validation_steps=50, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
100/100 [==============================] - 29s 88ms/step - loss: 0.7353 - accuracy: 0.4819 - val_loss: 0.6744 - val_accuracy: 0.5690
Epoch 2/20
100/100 [==============================] - 8s 84ms/step - loss: 0.6782 - accuracy: 0.5621 - val_loss: 0.6926 - val_accuracy: 0.5030
Epoch 3/20
100/100 [==============================] - 8s 83ms/step - loss: 0.6894 - accuracy: 0.5154 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 4/20
100/100 [==============================] - 8s 83ms/step - loss: 0.6906 - accuracy: 0.5095 - val_loss: 0.6903 - val_accuracy: 0.6090
Epoch 5/20
100/100 [==============================] - 9s 86ms/step - loss: 0.6896 - accuracy: 0.5399 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/20
100/100 [==============================] - 9s 86ms/step - loss: 0.6937 - accuracy: 0.5082 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/20
100/100 [==============================] - 9s 85ms/step - loss: 0.6933 - accuracy: 0.4979 - val_loss: 0.6931 - val_accuracy: 0.500

In [ ]:
model.save('model.h5')
save_history(history, 'history.bin')

In [ ]:
history = load_history('history.bin')
plot_compare(history)